In [2]:
import json
import pandas as pd
import re

dict_mapping = {'Loaded Language': 0,
 'Slogans': 1,
 'Reductio ad hitlerum': 2,
 'Causal Oversimplification': 3,
 'Thought-terminating cliché': 4,
 'Bandwagon': 5,
 'Repetition': 6,
 'Appeal to authority': 7,
 'Flag-waving': 8,
 'Obfuscation, Intentional vagueness, Confusion': 9,
 'Glittering generalities (Virtue)': 10,
 "Misrepresentation of Someone's Position (Straw Man)": 11,
 'Doubt': 12,
 'Name calling/Labeling': 13,
 'Presenting Irrelevant Data (Red Herring)': 14,
 'Exaggeration/Minimisation': 15,
 'Black-and-white Fallacy/Dictatorship': 16,
 'Smears': 17,
 'Appeal to fear/prejudice': 18,
 'Whataboutism': 19,
 'Transfer' : 20,
 'Appeal to (Strong) Emotions': 21
}


def dataset_f(data,dict_mapping):    
    data_id = []
    for element in data:
        dict_x = {}
        dict_x["text"] = element["text"]
        dict_x["labels"] = element["labels"]
        dict_x["id"] = element["id"]
        if len(element['labels']):
            data_id.append(dict_x)
    
    unprocessed = []
    processed = []
    labels = []
    ID = []
    for element in data_id:
        cleaned_text = re.sub(r'&', 'and',element['text'])
        pattern = r'\\n'
        cleaned_text = re.sub(pattern, ' ',cleaned_text)
        pattern = r'[^a-zA-Z0-9\s]+'
        cleaned_text = re.sub(pattern, '', cleaned_text).lower()
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
        if(cleaned_text != ''):
            unprocessed.append(element['text'])
            processed.append(cleaned_text)
            labels.append(element['labels'])
            ID.append(element['id'])
    
    data = {'text': processed, 'emo': labels,'id':ID}
    df = pd.DataFrame(data)

    for index,row in df.iterrows():
        l_labels = row['emo']
        empty_list = []
        for label in l_labels:
            empty_list.append(dict_mapping[label])
        row['emo'] = empty_list

    return df
    
with open(r'train.json', 'rb') as json_file:
    data = json.load(json_file)

df_train = dataset_f(data,dict_mapping)

with open(r'validation.json', 'rb') as json_file:
    data = json.load(json_file)

df_val = dataset_f(data,dict_mapping)

df_train_full = pd.concat([df_train,df_val],axis=0)

def list_to_string(lst):
    return ','.join(map(str, lst))

df_train_full['emo'] = df_train_full['emo'].apply(lambda x: list_to_string(x))
# df_train_full.to_csv("CDP_Complete_train_2b.tsv", sep='\t', index=False,header=False)

In [6]:
df_val['emo'] = df_val['emo'].apply(lambda x: list_to_string(x))
df_val.to_csv("CDP_Complete_val.tsv", sep='\t', index=False,header=False)

In [8]:
df = pd.read_csv('Merged_test_for_HypEmo_2b.csv').drop(['Unnamed: 0','text','image_ID'],axis=1)
df.rename(columns={'aug_text':'text','label':'emo'}, inplace=True)
df["emo"] = '9,6'
df
df.to_csv("CDP_Complete_test_2b.tsv", sep='\t', index=False,header=False)